In [1]:
# Importing the necessary libraries
import sqlite3
import pandas as pd

For the following excerecise we will be using sqlite3 to interact with SQL.

You will see different functions and methods.

The connect() function returns a connection object that we will use to interact with the SQLite database held in the file olistdb.

To execute SQL statements and fetch results from SQL queries, we will need to use a database cursor, the .cursor().

Then, we can write the code to be executed in the .execute() method. We will focus our attention on the content of these queries, as these can be used not only with sqlite3 (it can be used on an .sql file, among others).

Finally, to return the results we need to use the fetchall() method.

In [2]:
!gdown "153roT3S8vI2p07ok0xJHn8qq8S22mxHK"

# Creating the connection object
conn = sqlite3.connect('olist.db')

# Creating the cursor
c = conn.cursor()

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=153roT3S8vI2p07ok0xJHn8qq8S22mxHK 



First, we are going to calculate the amount of sales per state, all present in the same dataset, olist customers. Every order has the name of the state where it happened, so we will be counting them, grouping them and ordering them in descending order in the following query:

In [3]:
# Executing the query
c.execute('''
        SELECT customer_state,
        COUNT (customer_state) AS Amount
        FROM olist_customers
        GROUP BY customer_state
        ORDER BY Amount DESC
        ''')

# Fetching the results and creating a Dataframe with them
print (pd.DataFrame(c.fetchall(), columns=['Customer State',
                                           'Amount of Sales per State']))

OperationalError: no such table: olist_customers

Now, we are going to calculate the average review scoring of the orders per month and year. We will use the database called olist order reviews, that contains the review creation date (the date when the review was created) and the review score (from 1 to 5). For this we will have to select the month and the year of the review dates, and calculate the average of the review score, and group by the month of each year in the following query:

In [ ]:
# Executing the query
c.execute('''
        WITH review_time AS
        (SELECT
	        strftime('%m', review_creation_date) AS month_number,
	        strftime('%Y', review_creation_date) AS year,
                review_score
        FROM olist_order_reviews)
        SELECT month_number,
        AVG(CASE WHEN year = '2016' THEN review_score END),
        AVG(CASE WHEN year = '2017' THEN review_score END),
        AVG(CASE WHEN year = '2018' THEN review_score END)
        FROM review_time
        GROUP BY month_number;
        ''')

# Fetching the results and creating a Dataframe with them
print (pd.DataFrame(c.fetchall(),
                    columns=['Month',
                             '2016 Review Avg Score',
                             '2017 Review Avg Score',
                             '2018 Review Avg Score']))

Finally, we are going to calculate the total amount of delivered orders per city (top 15). For this we will have to join two datasets that contain all the info we want. Olist orders has the status of the orders and olist customers has the customer city. They both share a column called customer_id, that we will use to join them. We will count the number of cities, we will filter those that only have the "delivered" status, group by the customer cities and order everything by descending amount:

In [ ]:
# Executing the query
c.execute('''
        SELECT customer_city,
        COUNT (customer_city) AS Amount
        FROM olist_customers
        JOIN olist_orders USING (customer_id)
        WHERE order_status = 'delivered'
        GROUP BY customer_city
        ORDER BY Amount DESC
        LIMIT 15;
        ''')

# Fetching the results and creating a Dataframe with them
print (pd.DataFrame(c.fetchall(), columns=['Customer City',
                                           'Amount of Delivered Orders']))